<a href="https://colab.research.google.com/github/vinitakawale/Underwater-Image-Restoration/blob/main/UIRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/vinitakawale/Underwater-Image-Restoration.git

Cloning into 'Underwater-Image-Restoration'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 55 (delta 5), reused 46 (delta 3), pack-reused 0
Unpacking objects: 100% (55/55), done.


In [2]:
# py libs
import os
import time
import argparse
import numpy as np
from PIL import Image
from glob import glob
from ntpath import basename
from os.path import join, exists

In [3]:
# pytorch libs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
import torchvision.transforms as transforms

In [10]:
%cd /content/Underwater-Image-Restoration/

/content/Underwater-Image-Restoration


In [11]:
## options
opt = {"data_dir": "data/test/A/",
      "sample_dir":"data/output/",
      "model_name":"funiegan",
      "model_path":"models/funie_generator.pth"}

In [12]:
## checks
assert exists(opt["model_path"]), "model not found"
os.makedirs(opt["sample_dir"], exist_ok=True)
is_cuda = torch.cuda.is_available()
Tensor = torch.cuda.FloatTensor if is_cuda else torch.FloatTensor 

In [13]:
## model arch
if opt["model_name"].lower()=='funiegan':
    from nets import funiegan
    model = funiegan.GeneratorFunieGAN()
else: 
    # other models
    pass

In [14]:
## load weights
model.load_state_dict(torch.load(opt["model_path"]))
if is_cuda: model.cuda()
model.eval()
print ("Loaded model from %s" % (opt["model_path"]))

Loaded model from models/funie_generator.pth


In [15]:
# data pipeline
img_width, img_height, channels = 256, 256, 3
transforms_ = [transforms.Resize((img_height, img_width), Image.BICUBIC),
               transforms.ToTensor(),
               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]
transform = transforms.Compose(transforms_)

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [16]:
## testing loop
times = []
test_files = sorted(glob(join(opt["data_dir"], "*.*")))
for path in test_files:
    inp_img = transform(Image.open(path))
    inp_img = Variable(inp_img).type(Tensor).unsqueeze(0)
    # generate enhanced image
    s = time.time()
    gen_img = model(inp_img)
    times.append(time.time()-s)
    # save output
    img_sample = torch.cat((inp_img.data, gen_img.data), -1)
    save_image(img_sample, join(opt["sample_dir"], basename(path)), normalize=True)
    print ("Tested: %s" % path)

Tested: data/test/A/1.jpg
Tested: data/test/A/10.jpg
Tested: data/test/A/11.jpg
Tested: data/test/A/12.jpg
Tested: data/test/A/13.jpg
Tested: data/test/A/14.jpg
Tested: data/test/A/15.jpg
Tested: data/test/A/16.jpg
Tested: data/test/A/17.jpg
Tested: data/test/A/18.jpg
Tested: data/test/A/19.jpg
Tested: data/test/A/2.jpg
Tested: data/test/A/20.jpg
Tested: data/test/A/21.jpg
Tested: data/test/A/22.jpg
Tested: data/test/A/23.jpg
Tested: data/test/A/3.jpg
Tested: data/test/A/4.jpg
Tested: data/test/A/5.jpg
Tested: data/test/A/6.jpg
Tested: data/test/A/7.jpg
Tested: data/test/A/8.jpg
Tested: data/test/A/9.jpg
